### Imports

In [58]:
#import import_ipynb
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
#from baseline_models import get_words, get_nouns, get_noun_pairs, get_used_noun_frequency

### old imports from base line models
import pandas as pd
import numpy as np
import ast
import random
import re

from collections import Counter, defaultdict 

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt

import inflect
p = inflect.engine()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thrdl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Code

In [63]:
def prepare_text_keep_posts(df):
    texts = df.text.to_list() 
    
    posts = []
    for sublist in texts:
        sentences = []
        sentence = []
        for word in sublist:
            word = word.lower()
            if "n't" in word:
                word = "not"
            pattern = re.compile(r'[\"\'`,;]')
            if pattern.findall(word):
                pass
            else:
                pattern = re.compile(r'[\.\?\!]')
                if pattern.findall(word):
                    if len(sentence) > 1:
                        sentences.append(" ".join(sentence))
                    sentence = []
                else:
                    flag = True
                    for char in word:
                        if ord(char) > 127:
                            flag = False
                            break
                    if flag:
                        sentence.append(word)
        if len(sentence) > 0:
            sentences.append(" ".join(sentence))
        posts.append(sentences)

    return posts

def prepare_text(df):
    texts = df.text.to_list() 
    
    sentences = []
    for sublist in texts:
        sentence = []
        for word in sublist:
            word = word.lower()
            if "n't" in word:
                word = "not"
            pattern = re.compile(r'[\"\'`]')
            if pattern.findall(word):
                pass
            else:
                pattern = re.compile(r'[\.\?\!;,]')
                if pattern.findall(word):
                    if len(sentence) > 1:
                        sentences.append(" ".join(sentence))
                    sentence = []
                else:
                    flag = True
                    for char in word:
                        if ord(char) > 127:
                            flag = False
                            break
                    if flag:
                        sentence.append(word)
        if len(sentence) > 0:
            sentences.append(" ".join(sentence))

    return sentences

def tensor_label_array(embed,arrays_of_sentences_1, arrays_of_sentences_2):
    tensor_label = []
    for embedding, sentence in zip(embed, arrays_of_sentences_1):
        if sentence in arrays_of_sentences_2:
            tensor_label.append([embedding,1])
        else:
            tensor_label.append([embedding,0])
    #print(tensor_label[:5])
    return tensor_label

def get_words(df):
    sentences = df.text.to_list()    
    flat_list = []
    for sublist in sentences:
        for item in sublist:
            flat_list.append(item)           
    return flat_list

def get_noun_pairs(words):
    nouns = []
    noun_pairs = []
    porter = PorterStemmer()
    for i, (word, tag) in enumerate(words):
        if "n't" in word:
                word = "not"
        
        flag = True
        for char in word:
            if ord(char) > 127:
                flag = False
                break
        if len(word) >= 3 and tag.startswith('NN') and flag:
            if i > 0 and tag.startswith('NN') and (words[i-1][1].startswith('NN') or words[i-1][1].startswith('JJ') or words[i-1][1].startswith('PRP')):
                w1 = word
                w0 = ""
                for char in words[i-1][0]:
                    if ord(char) < 128:
                        w0 += char
                noun_pairs.append(w0+" "+w1)

            else:
                nouns.append(word)
                #nouns.append(porter.stem(word)) #convert words to their basic form
    return nouns, noun_pairs

def evaluate(tensor_label, df, label):
    accuracy = 0
    
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="default",
                           trainable=False)
    m = tf.keras.metrics.CosineSimilarity(axis=0)
    
    ctn = 0
    for row in df.iterrows():
        print(f"word: {ctn}")
        ctn +=1
        
        words_1 = get_words(val_1)
        tags_1 = nltk.pos_tag(words_1)
        nouns_1, pairs_1 = get_noun_pairs(tags_1)
        total_len = 0
        
        if len(pairs_1) > 0:
            mixture = np.array(pairs_1, dtype=bytes)
            total_len = len(pairs_1)
        else:
            mixture = np.array(nouns_1, dtype=bytes)
            total_len = len(nouns_1)
        
    
        embed_all = hub_layer(mixture)
        embed_all = tf.slice(embed_all, [0, 0], [total_len, 300])
        
        vote = 0
        for embed in embed_all:
            cos_max = -1
            label_vote = 0
            for tensors in tensor_label:
                m = tf.keras.metrics.CosineSimilarity(axis=0)
                m.update_state(embed, tensors[0])
                new_cos = m.result().numpy()
                if new_cos > cos_max:
                    cos_max = new_cos
                    label_vote = tensors[1]
            if label_vote == 0:
                vote -= 1
            else:
                vote += 1
        if (label == 0 and vote < 0) or (label == 1 and vote >= 0):
            accuracy += 1
    return accuracy

def evaluate_np(tensor_label, df, label):
    accuracy = 0
    posts = prepare_text_keep_posts(df)
    
    ctn = 0
    for post in posts:
        print(f"ctn: {ctn}")
        ctn += 1
        vote = 0
        
        embeds = get_all_embeddings(post,label)
        
        for embed in embeds:
            cos_max = -1
            label_vote = 0
            index = 0
            
            for j, tensors in enumerate(tensor_label):
                m = tf.keras.metrics.CosineSimilarity(axis=0)
                m.update_state(embed[0], tensors[0])
                new_cos = m.result().numpy()
                if new_cos > cos_max:
                    cos_max = new_cos
                    label_vote = tensors[1]
                    index = j
            if label_vote == 0:
                vote -= 1
            else:
                vote += 1
            if cos_max > 0.25:
                print(f"{cos_max}, {embed[2]} : {tensor_label[index][2]}")
                print("----------")
        if (label == 0 and vote < 0) or (label == 1 and vote >= 0):
            accuracy += 1
    
    return accuracy

def get_embedding_of_noun_pairs(sentence, label, model, hub_layer):
    words_to_tag = sentence.split()
    words = nltk.pos_tag(words_to_tag)
    noun_pair_positions = []    
    noun_pairs = []
    
    for i, (word, tag) in enumerate(words):
        if len(word) >= 3 and tag.startswith('NN'):
            if i > 0 and tag.startswith('NN') and (words[i-1][1].startswith('NN') or words[i-1][1].startswith('JJ') or words[i-1][1].startswith('PRP')):
                w1 = word
                w0 = ""
                for char in words[i-1][0]:
                    if ord(char) < 128:
                        w0 += char
                noun_pairs.append(w0+" "+w1)
                noun_pair_positions.append([i-1, i])
                
    
    #print(sentence)
    
    mixture = np.array([sentence], dtype=bytes)
    #print(mixture)
    
    embed = hub_layer(mixture)
    
    noun_embeddings = []
    for i, pair in enumerate(noun_pair_positions):
        t1 = tf.slice(embed[0][pair[0]], [0], [50])
        t2 = tf.slice(embed[0][pair[1]], [0], [250])
        t_res = tf.concat([t1, t2],0) 
    
        noun_embeddings.append([t_res, label, noun_pairs[i]])
    return noun_embeddings
    
    
def get_all_embeddings(sentences, label):
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="elmo",
                           trainable=False)
    
    noun_embeddings = []
    for i, sentence in enumerate(sentences[:]):
        if i % 100 == 0:
            print(f"s_{i}")
        noun_embeddings.extend(get_embedding_of_noun_pairs(sentence, label, model, hub_layer))
    return noun_embeddings

In [68]:
topics = { "abortion": ["abortion_pro_choice.csv", "abortion_pro_life.csv"],
           "gay_marriage":["gay_marriage_for.csv", "gay_marriage_against.csv"],
           "darwin_theory_of_evolution" :["darwin_theory_of_evolution_for.csv", "darwin_theory_of_evolution_against.csv"],
          "marijuana_legalization" :["marijuana_legalization_against.csv", "marijuana_legalization_for.csv"],
         }


for key, pair in topics.items():

    df_1 = pd.read_csv(f"./dataset_processed/{pair[0]}", converters={2:ast.literal_eval})
    df_2 = pd.read_csv(f"./dataset_processed/{pair[1]}", converters={2:ast.literal_eval})
    
    
    tr_1, val_1 = train_test_split(tr_1, test_size=0.25, random_state=25)
    tr_2, val_2 = train_test_split(tr_2, test_size=0.25, random_state=25)
    
    arrays_of_sentences_1 = prepare_text(tr_1)
    arrays_of_sentences_2 = prepare_text(tr_2)
    
    embeddings_1  = get_all_embeddings(arrays_of_sentences_1, 0)
    embeddings_2  = get_all_embeddings(arrays_of_sentences_2, 1)
    
    #_1 extended with _2 = not needed
    embeddings_1.extend(embeddings_2)
    print(len(embeddings_1))
    
    acc_1 = evaluate_np(embeddings_1, val_1, 0)
    acc_2 = evaluate_np(embeddings_1, val_2, 1)
    
    print(f"validation err: {(acc_1+acc_2)/(len(val_1.index)+len(val_2.index))}")
    """
    
    arrays_of_sentences_1.extend(arrays_of_sentences_2)
    random.shuffle(arrays_of_sentences_1)
    
    
    #mixture = arrays_of_sentences_1
    mixture = np.array(arrays_of_sentences_1, dtype=bytes)
    
    print(len(mixture))

    
    model = "https://tfhub.dev/google/elmo/3"
    hub_layer = hub.KerasLayer(model, output_key="default",
                           trainable=False)
    
    embed_all = hub_layer(mixture[:100])
    embed_all = tf.slice(embed_all, [0, 0], [100, 300]) 
    
    for i in range(100, len(mixture)-200, 200):
        print(i)
        embed = hub_layer(mixture[i:i+200])
        embed = tf.slice(embed, [0, 0], [200, 300]) 
        embed_all = tf.concat([embed_all, embed], 0) 
    print("executed")
    tensor_label = tensor_label_array(embed,arrays_of_sentences_1, arrays_of_sentences_2)
    
    acc_1 = evaluate(tensor_label, val_1, 0)
    acc_2 = evaluate(tensor_label, val_2, 1)
    
    print(f"validation err: {(acc_1+acc_2)/(len(val_1.index)+len(val_2.index))}")
    """

### Noun distance

In [53]:
i_A = 0
i_B = 0
m = tf.keras.metrics.CosineSimilarity(axis=0)
for i, emb in enumerate(embeddings_1):
    last_word = emb[2].split()
    if last_word[1].startswith("murder"):
        i_A = i
    if last_word[1].startswith("abortion"):
        i_B = i
    if i_A != 0 and i_B != 0:
        break

m.update_state(embeddings_1[i_A][0], embeddings_1[i_B][0])
print(f"cos dist {embeddings_1[i_A][2]} + {embeddings_1[i_B][2]}: {m.result().numpy()}")

cos dist is murder + if abortion: 0.3986886143684387
